In [2]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import joblib


# Data Reading
file_path = 'augmented_datasetnew.csv'
data = pd.read_csv(file_path)

# Data cleaning
data['Text'] = data['Text'].astype(str).str.lower()
data['Text'] = data['Text'].str.replace(r'[^\w\s]', '', regex=True)
# Disease Label Coding
label_encoder = LabelEncoder()
data['Disease label'] = label_encoder.fit_transform(data['Disease name'])

# Dividing the data into training and testing.
X = data['Text']
y = data['Disease label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Converting texts to numerical representation using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Training a Logistic Regression Model
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train_tfidf, y_train)

# Training SVM Model
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train_tfidf, y_train)

# Training the XGBoost model
xgboost_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgboost_model.fit(X_train_tfidf, y_train)

# Integrating models using Voting Classifier
voting_model = VotingClassifier(
    estimators=[
        ('Logistic Regression', logistic_model),
        ('SVM', svm_model),
        ('XGBoost', xgboost_model)
    ],
    voting='soft'
)
voting_model.fit(X_train_tfidf, y_train)

# Model Evaluation
models = {
    "Logistic Regression": logistic_model,
    "SVM": svm_model,
    "XGBoost": xgboost_model,
    "Voting Classifier": voting_model
}

for name, model in models.items():
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy * 100:.2f}%")
    print(f"{name} Classification Report:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Save the best model (Voting Classifier)
joblib.dump(voting_model, 'voting_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
print("The model and tools have been saved successfully!")


# Running the interface in the kernel
if __name__ == "__main__":
    print("Welcome to the symptom-based disease diagnosis system.")
    print("Please enter a description of symptoms to get the expected diagnosis.")

    # Download the templates
    voting_model = joblib.load('voting_model.pkl')
    tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
    label_encoder = joblib.load('label_encoder.pkl')

    while True:
        user_symptom = input("\nEnter a description of the symptoms (or type 'exit' to exit): ").strip()
        if user_symptom.lower() == 'exit':
            print("Thank you for using the system. Farewell!")
            break

        # Obtaining the probabilities from all models.
        input_vector = tfidf_vectorizer.transform([user_symptom])
        total_probabilities = voting_model.predict_proba(input_vector)[0]

        # Determining the predicted disease based on the highest probability.
        max_idx = np.argmax(total_probabilities)
        predicted_disease = label_encoder.inverse_transform([max_idx])[0]

        # Calculation of probability ratios for each model
        individual_model_probs = {}
        for model in voting_model.estimators_:
            model_prob = model.predict_proba(input_vector)[0]
            model_name = model.__class__.__name__
            individual_model_probs[model_name] = model_prob

        # Determining the model that contributed the highest probability.
        max_contribution_model = max(individual_model_probs, key=lambda x: individual_model_probs[x][max_idx])

        # Display the results
        print(f"\nPredicted Disease: {predicted_disease}")
        print(f"Probability: {total_probabilities[max_idx] * 100:.2f}%")
        print(f"Model Contributing Most to Prediction: {max_contribution_model}\n")

        # Display the odds ratios for each disease with each model.
        print("Probabilities by each model:")
        for model_name, probs in individual_model_probs.items():
            print(f"{model_name}:")
            for idx, prob in enumerate(probs):
                disease_name = label_encoder.inverse_transform([idx])[0]
                print(f"  - {disease_name}: {prob * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:04:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:04:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Logistic Regression Accuracy: 96.77%
Logistic Regression Classification Report:
                           precision    recall  f1-score   support

                     Acne       1.00      1.00      1.00        13
       Contact Dermatitis       1.00      0.77      0.87        13
                   Eczema       1.00      1.00      1.00        13
                 Impetigo       0.82      1.00      0.90        14
                Psoriasis       1.00      1.00      1.00        13
Ringworm (Tinea Corporis)       1.00      1.00      1.00        13
                  Scabies       1.00      1.00      1.00        14

                 accuracy                           0.97        93
                macro avg       0.97      0.97      0.97        93
             weighted avg       0.97      0.97      0.97        93

SVM Accuracy: 100.00%
SVM Classification Report:
                           precision    recall  f1-score   support

                     Acne       1.00      1.00      1.00       